# Toolkit Showcase: Prepare and run a protein-ligand simulation

<br />
<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>

    # The simplest way to install dependencies is to install the examples package:
    
    conda install -c conda-forge openff-toolkit-examples
    
    # This example will need access to a GROMACS install in addition to the above dependencies. Your existing GROMACS installed can be used, or you can install it from Bioconda:
    
    conda install -c conda-forge -c bioconda gromacs
    
    # You can also install all the depencies using the provided environment.yaml:
    
    conda env update --file ../environment.yaml 
    
    # You may also need to restart this notebook's kernel after you make these changes (Kernel -> Restart)
</details>


Welcome to the Open Force Field Toolkit! This introductory example will demonstrate how to prepare a system for Molecular Dynamics that combines solvent, a ligand using the Parsley open force field, and a protein using a standard Amber force field. We'll use a number of tools from the Python molecular science ecosystem to take a pre-docked benchmark example from scratch all the way through to simulation and visualization, all without leaving the notebook. Have fun!

In [1]:
%%html
<!--" Stylesheet for improved readability. This example has a lot of reading, so we want good typography. "-->
<style>
p, ul, ol, h1, h2, h3, h4, h5, h6 {
    max-width: 50rem;    
}
</style>

In [2]:
# Imports from the Python standard library
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

# Imports from dependencies
from simtk import openmm, unit
import parmed as pmd
import numpy as np
import mdtraj as mdt
import nglview
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

# Imports from the toolkit
import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.topology import Molecule, Topology

# Imports from local files
from utils import off_topology_to_omm

_(The OpenEye loading warning is expected -- The toolkit is informing us that OETK is unavailable, but it will safely fall back to using RDKit and AmberTools for the same functionality)_

## Introducing the main cast

Merck [provides data](https://github.com/MCompChem/fep-benchmark) to benchmark Free Energy Perturbation (FEP) procedures. We'll use structures from this dataset for this showcase:

https://github.com/MCompChem/fep-benchmark

This example is pre-packaged with one protein-ligand complex from the above repository, however you should be able to download other complexes and run them as well using this workflow. CHEMBL1078774, our ligand of choice, is an inhibitor of the mitotic functions of kinesin-5, a motor protein involved in cell division. 

The ligand and protein structures are already prepared for simulation:

- Their co-ordinates are super-imposable
- Hydrogens have been added to protein and crystallographic waters
- The protein's termini are capped with N-methyl and acetyl groups to prevent unphysical charges
- Atoms missing from the crystal structure have been replaced

<br />
<details>
    <summary><small>▼ Click here for the shell commands we used to download the protein-ligand complex</small></summary>

    # Clone the repository
    git clone https://github.com/MCompChem/fep-benchmark.git
    # Take the first ligand from the eg5 benchmark
    head -n119 fep-benchmark/eg5/ligands.sdf > chembl_1078774.sdf
    # Take the prepared protein structure
    cp fep-benchmark/eg5/3l9h_prepared.pdb .
    
</details>

In [3]:
receptor_path = "3l9h_prepared.pdb"
ligand_path = "chembl_1078774.sdf"

We can visualize each structure using the [NGLView](https://github.com/nglviewer/nglview) widget. These visualizations are interactive; rotate by dragging the left mouse button, pan with the right mouse button, and zoom with the scroll wheel. You can also mouse over an atom to see its details, and click an atom to center the view on it. When you mouse over the widget, a full screen button will appear in its top right corner. 

In [4]:
view = nglview.show_file(ligand_path)
view

NGLWidget()

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ Try replacing <code>ligand_path</code> with <code>receptor_path</code> to visualize the protein!
</div>


# The plan:

| Action | Software|
|--|--|
| Assemble the force field | OpenFF Toolkit, OpenMM Force Fields, OpenMM
| Model and solvate the complex | OpenMM
| Parameterize the complex | OpenMM (OpenFF Toolkit is under the hood)
| Visualize the complex | ParmEd, NGLView
| Simulate the complex | OpenMM
| Visualize the simulation | NGLView


## Assemble the force field (OpenFF Toolkit, OpenMM Force Fields, OpenMM)

The first step in this workflow, as with any simulation, is to choose the force field. The OpenMM Force Fields package ([`openmmforcefields`](https://github.com/openmm/openmmforcefields)) allows us to combine force fields for different components so that we can model the system in one go.

The star of the show here is the Parsley 1.3.0 force field. [Parsley](https://openforcefield.org/force-fields/force-fields/#parsley) is the first generation force field produced by the Open Force Field Initiative. Rather than using atom types like traditional biomolecular force fields, Parsley assigns parameters to a molecule with [fancy subgraph matching](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html). Note that to parameterize a molecule you need more than just the co-ordinates of its atoms; you also need their bonds, bond orders, and formal charges. As a result, `.sdf` files are used in this example; other file types are possible, but they must include this information. Parsley is distributed as an `.offxml` file according to the [SMIRNOFF specification](https://open-forcefield-toolkit.readthedocs.io/en/latest/smirnoff.html). This file describes all the choices the toolkit has to make to parameterize a molecule.

The Parsley force field is designed for small molecule parameterization. Parameters for proteins and other polymers are coming, but for now we'll use Amber 99sb as our protein force field. The protein is already prepared with Amber-compatible residue names, so this is simple. Finally, we'll use the TIP3p water model, as both Parsley and Amber are parameterized for it.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that OpenMM and the Open Force Field Toolkit both have classes called `Topology` and `ForceField` that serve similar functions. Don't get them confused!
</div>


In [5]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)
# Molecule stores both the co-ordinates of the atoms and their bond graph
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()

# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField(
    "amber99sb.xml", "tip3p.xml"
)
# Load the ligand molecule and the Parsley force field
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
omm_forcefield.registerTemplateGenerator(smirnoff.generator)

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ This is the only block in the example that uses the Open Force Field Toolkit directly!
</div>

## Model and solvate the complex (OpenMM)

Conceptually, this step involves putting together the positions of all of the components of the system. In practice, this is also where the Parsley parameters for the ligand are actually generated. This involves computing partial charges for its atoms, which takes time.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that 4 Å is not enough padding for regular simulations; we're just using it for a fast example.
</div>

In [6]:
# Load the kinesin-5 receptor structure into Modeller
pdb = openmm.app.PDBFile(receptor_path)
modeller = openmm.app.Modeller(pdb.topology, pdb.positions)

# Add the ligand
# Under the hood, this line uses the OpenFF Toolkit to generate new parameters for the ligand
modeller.add(off_topology_to_omm(ligand_topology), ligand_positions)

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * unit.angstrom,
    ionicStrength=0.15 * unit.molar,
)


_(This'll take a minute or two)_

In this workflow, the Toolkit is just responsible for combining a force field with a molecular topology, and much of this work happens behind the scenes through the OpenMM ForceField object. The toolkit is designed to check the input and give the user useful feedback if there seem to be errors or problems in the provided files. It also computes partial charges without user intervention. Computing charges is a process that can be confusing and error-prone, so we try to specify and simplify it as much as possible. Charges are computed efficiently with [OpenEye](https://www.eyesopen.com/) if is available; if it is not, the free toolkits [RDKit](https://www.rdkit.org/) and [AmberTools](https://ambermd.org/AmberTools.php) are used instead.

    
### Charge generation

So far, released OpenFF force fields only use AM1-BCC partial charge generation. Other semi-empirical methods and charge corrections are available in the toolkit, and they'll be used in future force fields. In particular, so-called "graph based" charges are coming soon! Graph-based charges use only the bond graph to compute charges, rather than requiring 3D coordinates. This promises to make the produced partial charges independent of the conformation (a feature we currently provide at [some cost](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-dont-seem-to-depend-on-the-molecules-conformation-is-this-a-bug)), and may even be a lot faster to boot.

The toolkit also supports [library charges](https://open-forcefield-toolkit.readthedocs.io/en/latest/api/generated/openff.toolkit.typing.engines.smirnoff.parameters.LibraryChargeHandler.html#openff.toolkit.typing.engines.smirnoff.parameters.LibraryChargeHandler) that can be assigned via a template.
    

## Parameterize the complex (OpenMM, OpenFF Toolkit under the hood)

This step assigns parameters to the solvent and protein, and combines them with the ligand parameters into an OpenMM `System` object, which encodes all the forces and interactions in the molecular system.

In [7]:
# Construct the OpenMM System, which stores the forces for the system
system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

# Retrieve the OpenMM Topology, which stores the atoms and connectivity
topology = modeller.getTopology()

# Get the initial positions
# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
positions = modeller.getPositions() + np.array([30, 30, 30]) * unit.angstrom

## Visualize the complex (ParmEd, NGLView)

[ParmEd](https://github.com/ParmEd/ParmEd) is able to read, write, combine, and edit topologies and structures from several different simulation programs, including CHARMM, OpenMM, Amber, and GROMACS. Here, we'll use it to convert our OpenMM `System` to a format we can visualize, and later we'll use it to convert our prepared system to another simulation package.

ParmEd doesn't support constraints from OpenMM because different simulation packages have conflicting ways of specifying them. So we need to create a new OpenMM `System` object here, and read it into a ParmEd [`Structure`](https://parmed.github.io/ParmEd/html/structure.html) (see also [API docs](https://parmed.github.io/ParmEd/html/structobj/parmed.structure.Structure.html))

In [8]:
# ParmEd can't handle constraints from openmmforcefields, so we need a variant for export without them
export_system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    constraints=None,
    rigidWater=False,
)

# Combine the topology, system and positions into a ParmEd Structure
pmd_complex_struct = pmd.openmm.load_topology(
    topology, export_system, positions
)

To visualize inside the notebook, we'll use NGLView. NGLView supports a wide variety of [molecular visualization methods](https://nglviewer.org/ngl/api/manual/molecular-representations.html), as well as a VMD-like [atom selection language](https://nglviewer.org/ngl/api/manual/selection-language.html). This can be used to visualize complex systems like this one.

The widget consists of a minimally documented [Python library frontend](https://nglviewer.org/nglview/latest/api.html) and an extensively documented [JavaScript backend](https://nglviewer.org/ngl/api/manual/index.html). You'll need to refer to the documentation for both to do anything sophisticated, as the Python code delegates most of its options and functionality to the JS code.

In [9]:
# Create the widget. By default, proteins are shown as a cartoon and unrecognised ligands with a ball-and-stick model
view = nglview.show_parmed(pmd_complex_struct)
    
# Add a licorice/stick representation for everything except the protein
view.add_licorice(selection="(not protein)")
# Make the ions clearer by drawing their vdW surfaces
view.add_surface(selection=":.NA or :.CL")

# Render the widget
view

NGLWidget()

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ Have a play with this visualization! Try clearing the default representations with <code>view.clear()</code> and configuring your own cartoon <em>(Hint: <a href=https://nglviewer.org/nglview/latest/api.html#nglview.NGLWidget>Check</a> the <a href=https://nglviewer.org/ngl/api/manual/molecular-representations.html>docs</a>)</em>. See if you can display the ligand in a way you like, without also displaying the protein's terminal caps. When you're happy with what you've made, save the image with <code>view.download_image()</code>
</div>

## Simulate the complex (OpenMM)

All that remains is to tell OpenMM the details about how we want to integrate and record data for the simulation, and then to put everything together and run it! 

### Configure and run the simulation

Here, we'll use a Langevin thermostat at 300 Kelvin and a 2 fs time step. We'll write the structure to disk every 10 steps.

In [10]:
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(
    topology, system, integrator
)
simulation.context.setPositions(positions)

# Add a reporter to record the structure every 10 steps
nc_reporter = pmd.openmm.NetCDFReporter("trajectory.nc", 10)
simulation.reporters.append(nc_reporter)

### Minimize the combined system

This will reduce any forces that are too large to integrate, such as from clashes or from disagreements between the crystal structure and force field.


In [11]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True,
    getEnergy=True,
    getForces=True
)

print(
    "Minimised to", 
    minimized_state.getPotentialEnergy(), 
    "with maximum force", 
    max([np.sqrt(v.x*v.x + v.y*v.y + v.z*v.z) for v in minimized_state.getForces()]),
    minimized_state.getForces().unit.get_symbol()
)

minimized_coords = minimized_state.getPositions()

Minimised to -650770.7265697164 kJ/mol with maximum force 2497.7936058055598 kJ/(nm mol)


_(This'll take a minute or two)_

### Run a short simulation

If this were anything more than a demonstration of the Toolkit, this example would need to include additional steps like equilibration. 

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [12]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

/home/joshmitchell/conda/envs/offdev/lib/python3.7/site-packages/parmed/amber/netcdffiles.py:409: UserWarning: Could not find netCDF4 module. Falling back on scipy implementation, which can significantly slow down simulations if used as a reporter
  warnings.warn('Could not find netCDF4 module. Falling back on '


_(This'll take a minute or two, we're writing to disk very frequently)_

### Open Source Force Fields

A primary goal of the Open Force Field Initiative is to make development and use of force fields as open as possible - it's in our name! We believe that open source development practices have a lot to offer the scientific community, whether that science is academic, commercial, or hobbyist.

#### The SMIRNOFF specification

The SMIRNOFF specification describes a simple format for describing molecular force fields. We provide and maintain this spec in the hopes that it will allow scientists everywhere to contribute to force field development in a unified way, without taking them away from their favourite simulation package.

SMIRNOFF is not just a spec; we're also committed to a reference implementation — that being the OpenFF Toolkit. The Toolkit will always support all the functional forms in both the SMIRNOFF spec and the [`openff-forcefields`](https://github.com/openforcefield/openff-forcefields/) package.

#### Reproducibility

OpenFF force fields are completely specified by the name of the distributed `.offxml` file. We use codenames, version numbers, and tags to accomplish this. This means that as long as a user, designer, or reviewer sees the name of the force field being used, they know exactly what is going in to that simulation. We include parameters that are often treated as implementation details in this specification, so details like the cut-off distance, constraints, modifications to the Lennard-Jones function, and partial charge generation methods are all defined by the name of the force field. 

As much as possible, we want energy and force to be a deterministic output of combining a molecule and a force field. If an author provides the name of the force field in their methods section, it should be reproducible. The other side of this coin is that we never want to hide the force field from the user. In all our workflows, the name of the force field must be explicitly provided in the code. This improves reproducibility of the code and helps the user take responsibility for their results. 

#### "Plugin" support for new force fields

The OpenFF Toolkit supports distributing force field files (.offxml) through Conda data packages. Anyone can publish a package on Conda Forge that extends the list of directories the toolkit searches for force fields, allowing anyone to produce force fields without requiring their own tooling, in a format that is designed to be converted to a multitude of simulation packages. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#how-can-i-distribute-my-own-force-fields-in-smirnoff-format) for more details.


## Visualize the simulation (nglview)

NGLView can display trajectories, as well as single structures. Mouse over the widget to see the animation controls.

In [13]:
with open("system.pdb", "w") as f:
    openmm.app.PDBFile.writeFile(topology, positions, f)
mdt_traj = mdt.load("trajectory.nc", top="system.pdb")

view = nglview.show_mdtraj(mdt_traj)
view

NGLWidget(max_frame=99)

## Conclusions

* Toolkit parameterization of an unknown ligand slides right in alongside specifying the protein and solvent force fields
* Open source tools installed via Conda did everything, from basic system prep to simulation and visualization
* Using OpenMM, we never had to leave Python or touch the disk to set up the simulation
* With ParmEd, using GROMACS, Amber or CHARMM is simple! See the appendix for details.


## What's next?

We have more examples for you to explore the toolkit!

 - [Force Field Modification](https://github.com/openforcefield/openff-toolkit/blob/master/examples/forcefield_modification/ManipulateParameters.ipynb): Use the Toolkit API to manipulate SMIRNOFF parameters in a `ForceField` object
 - [Conformer energies](https://github.com/openforcefield/openff-toolkit/blob/master/examples/conformer_energies/conformer_energies.ipynb): Compute vacuum energies of different conformers of a small molecule with the Parsley force field
 - [QCArchive interface](https://github.com/openforcefield/openff-toolkit/blob/master/examples/QCArchive_interface/QCarchive_interface.ipynb): Create OpenFF `Molecule` objects from the [QCArchive](https://qcarchive.molssi.org/)

# Appendix: What about GROMACS?

OpenMM makes it easy to run molecular simulations without leaving Python. The OpenFF toolkit currently exports directly to OpenMM, but no part of the parametrization process is exclusively supported by OpenMM. With ParmEd and other tools, the same systems can be run in other engines. Here we show how to use ParmEd to prepare and run the same workflow in GROMACS.

We have [another example](https://github.com/openforcefield/openff-toolkit/blob/master/examples/using_smirnoff_in_amber_or_gromacs/convert_to_amber_gromacs.ipynb) that uses ParmEd to produce Amber input files too, and also to validate single point energies.

In [14]:
# Export GROMACS files.
pmd_complex_struct.save("system.top", overwrite=True)
pmd_complex_struct.save("system.gro", overwrite=True)

Energy minimize the system. Unfortunately the net charge is not zero; this is a [known bug](https://github.com/openforcefield/openff-toolkit/issues/456) in the toolkit.

In [15]:
# Configure GROMACS not to clutter up our directory with backups
%env GMX_MAXBACKUP=-1

# Energy minimise the coordinates
! gmx grompp -f minim.mdp -c system.gro -p system.top -o em.tpr -maxwarn 1
! gmx mdrun -v -deffnm em


env: GMX_MAXBACKUP=-1
                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meul

Step=   16, Dmax= 1.5e-02 nm, Epot= -4.86489e+05 Fmax= 2.28040e+03, atom= 1607
Step=   18, Dmax= 9.2e-03 nm, Epot= -4.88176e+05 Fmax= 1.00633e+04, atom= 1607
Step=   19, Dmax= 1.1e-02 nm, Epot= -4.94192e+05 Fmax= 5.24317e+03, atom= 1607
Step=   21, Dmax= 6.7e-03 nm, Epot= -4.96394e+05 Fmax= 3.76659e+03, atom= 1607
Step=   22, Dmax= 8.0e-03 nm, Epot= -4.98073e+05 Fmax= 7.30530e+03, atom= 1607
Step=   23, Dmax= 9.6e-03 nm, Epot= -5.00242e+05 Fmax= 5.66583e+03, atom= 1607
Step=   24, Dmax= 1.2e-02 nm, Epot= -5.00631e+05 Fmax= 1.02111e+04, atom= 1607
Step=   25, Dmax= 1.4e-02 nm, Epot= -5.02890e+05 Fmax= 8.50738e+03, atom= 1607
Step=   27, Dmax= 8.3e-03 nm, Epot= -5.05746e+05 Fmax= 2.79449e+03, atom= 1607
Step=   28, Dmax= 9.9e-03 nm, Epot= -5.05884e+05 Fmax= 1.04231e+04, atom= 1607
Step=   29, Dmax= 1.2e-02 nm, Epot= -5.09296e+05 Fmax= 7.67108e+03, atom= 2434
Step=   31, Dmax= 7.2e-03 nm, Epot= -5.11278e+05 Fmax= 3.06346e+03, atom= 2434
Step=   32, Dmax= 8.6e-03 nm, Epot= -5.12646e+05 Fma

Step=  147, Dmax= 2.0e-02 nm, Epot= -5.65242e+05 Fmax= 1.59980e+04, atom= 1019
Step=  149, Dmax= 1.2e-02 nm, Epot= -5.65882e+05 Fmax= 2.07393e+03, atom= 1019
Step=  150, Dmax= 1.4e-02 nm, Epot= -5.65971e+05 Fmax= 1.99576e+04, atom= 1019
Step=  151, Dmax= 1.7e-02 nm, Epot= -5.66852e+05 Fmax= 6.16623e+03, atom= 1019
Step=  153, Dmax= 1.0e-02 nm, Epot= -5.66956e+05 Fmax= 9.54392e+03, atom= 1019
Step=  154, Dmax= 1.2e-02 nm, Epot= -5.67136e+05 Fmax= 9.20562e+03, atom= 1019
Step=  155, Dmax= 1.5e-02 nm, Epot= -5.67142e+05 Fmax= 1.33731e+04, atom= 1019
Step=  156, Dmax= 1.8e-02 nm, Epot= -5.67293e+05 Fmax= 1.36358e+04, atom= 2031
Step=  158, Dmax= 1.1e-02 nm, Epot= -5.67776e+05 Fmax= 2.63526e+03, atom= 2031
Step=  159, Dmax= 1.3e-02 nm, Epot= -5.67815e+05 Fmax= 1.69157e+04, atom= 2031
Step=  160, Dmax= 1.5e-02 nm, Epot= -5.68423e+05 Fmax= 6.54370e+03, atom= 2031
Step=  162, Dmax= 9.2e-03 nm, Epot= -5.68569e+05 Fmax= 7.53814e+03, atom= 2031
Step=  163, Dmax= 1.1e-02 nm, Epot= -5.68689e+05 Fma

Step=  278, Dmax= 1.1e-02 nm, Epot= -5.84147e+05 Fmax= 9.44923e+03, atom= 2031
Step=  279, Dmax= 1.3e-02 nm, Epot= -5.84225e+05 Fmax= 9.98037e+03, atom= 2031
Step=  280, Dmax= 1.5e-02 nm, Epot= -5.84273e+05 Fmax= 1.33981e+04, atom= 2031
Step=  281, Dmax= 1.8e-02 nm, Epot= -5.84339e+05 Fmax= 1.45485e+04, atom= 2031
Step=  282, Dmax= 2.2e-02 nm, Epot= -5.84344e+05 Fmax= 1.90467e+04, atom= 2031
Step=  283, Dmax= 2.6e-02 nm, Epot= -5.84379e+05 Fmax= 2.10773e+04, atom= 2031
Step=  285, Dmax= 1.6e-02 nm, Epot= -5.84682e+05 Fmax= 3.07042e+03, atom= 2031
Step=  287, Dmax= 9.4e-03 nm, Epot= -5.84801e+05 Fmax= 1.14358e+04, atom= 2031
Step=  288, Dmax= 1.1e-02 nm, Epot= -5.84917e+05 Fmax= 6.01306e+03, atom= 2031
Step=  289, Dmax= 1.4e-02 nm, Epot= -5.84948e+05 Fmax= 1.48794e+04, atom= 2031
Step=  290, Dmax= 1.6e-02 nm, Epot= -5.85082e+05 Fmax= 1.02368e+04, atom= 2031
Step=  292, Dmax= 9.8e-03 nm, Epot= -5.85186e+05 Fmax= 4.80167e+03, atom= 2031
Step=  293, Dmax= 1.2e-02 nm, Epot= -5.85247e+05 Fma

Step=  408, Dmax= 1.1e-02 nm, Epot= -5.93944e+05 Fmax= 2.86791e+03, atom= 2031
Step=  409, Dmax= 1.3e-02 nm, Epot= -5.93983e+05 Fmax= 1.78621e+04, atom= 2031
Step=  410, Dmax= 1.6e-02 nm, Epot= -5.94156e+05 Fmax= 7.05381e+03, atom= 2031
Step=  412, Dmax= 9.7e-03 nm, Epot= -5.94206e+05 Fmax= 7.89421e+03, atom= 2031
Step=  413, Dmax= 1.2e-02 nm, Epot= -5.94247e+05 Fmax= 1.00506e+04, atom= 2031
Step=  414, Dmax= 1.4e-02 nm, Epot= -5.94288e+05 Fmax= 1.14752e+04, atom= 2031
Step=  415, Dmax= 1.7e-02 nm, Epot= -5.94310e+05 Fmax= 1.43290e+04, atom= 2031
Step=  416, Dmax= 2.0e-02 nm, Epot= -5.94331e+05 Fmax= 1.66316e+04, atom= 2031
Step=  418, Dmax= 1.2e-02 nm, Epot= -5.94499e+05 Fmax= 1.94027e+03, atom= 2031
Step=  419, Dmax= 1.4e-02 nm, Epot= -5.94598e+05 Fmax= 2.02789e+04, atom= 2031
Step=  420, Dmax= 1.7e-02 nm, Epot= -5.94816e+05 Fmax= 6.55720e+03, atom= 2031
Step=  422, Dmax= 1.0e-02 nm, Epot= -5.94855e+05 Fmax= 9.53967e+03, atom= 2031
Step=  423, Dmax= 1.2e-02 nm, Epot= -5.94906e+05 Fma

Step=  540, Dmax= 1.7e-02 nm, Epot= -6.00992e+05 Fmax= 1.48256e+04, atom= 2031
Step=  541, Dmax= 2.1e-02 nm, Epot= -6.01000e+05 Fmax= 1.69798e+04, atom= 2031
Step=  543, Dmax= 1.2e-02 nm, Epot= -6.01164e+05 Fmax= 2.13367e+03, atom= 2031
Step=  545, Dmax= 7.4e-03 nm, Epot= -6.01227e+05 Fmax= 9.38811e+03, atom= 2031
Step=  546, Dmax= 8.9e-03 nm, Epot= -6.01290e+05 Fmax= 4.40049e+03, atom= 2031
Step=  547, Dmax= 1.1e-02 nm, Epot= -6.01298e+05 Fmax= 1.21554e+04, atom= 2031
Step=  548, Dmax= 1.3e-02 nm, Epot= -6.01373e+05 Fmax= 7.69371e+03, atom= 2031
Step=  550, Dmax= 7.7e-03 nm, Epot= -6.01424e+05 Fmax= 4.21356e+03, atom= 2031
Step=  551, Dmax= 9.2e-03 nm, Epot= -6.01448e+05 Fmax= 1.00539e+04, atom= 2031
Step=  552, Dmax= 1.1e-02 nm, Epot= -6.01506e+05 Fmax= 7.08749e+03, atom= 2031
Step=  554, Dmax= 6.7e-03 nm, Epot= -6.01554e+05 Fmax= 3.19306e+03, atom= 2031
Step=  555, Dmax= 8.0e-03 nm, Epot= -6.01591e+05 Fmax= 9.18038e+03, atom= 2031
Step=  556, Dmax= 9.6e-03 nm, Epot= -6.01648e+05 Fma

Step=  673, Dmax= 1.3e-02 nm, Epot= -6.06184e+05 Fmax= 1.07777e+04, atom= 2031
Step=  675, Dmax= 7.9e-03 nm, Epot= -6.06259e+05 Fmax= 1.46499e+03, atom= 2031
Step=  676, Dmax= 9.5e-03 nm, Epot= -6.06326e+05 Fmax= 1.33277e+04, atom= 2031
Step=  677, Dmax= 1.1e-02 nm, Epot= -6.06426e+05 Fmax= 4.34789e+03, atom= 2031
Step=  679, Dmax= 6.8e-03 nm, Epot= -6.06453e+05 Fmax= 6.24500e+03, atom= 2031
Step=  680, Dmax= 8.2e-03 nm, Epot= -6.06483e+05 Fmax= 6.45372e+03, atom= 2031
Step=  681, Dmax= 9.9e-03 nm, Epot= -6.06500e+05 Fmax= 8.78800e+03, atom= 2031
Step=  682, Dmax= 1.2e-02 nm, Epot= -6.06525e+05 Fmax= 9.49306e+03, atom= 2031
Step=  683, Dmax= 1.4e-02 nm, Epot= -6.06528e+05 Fmax= 1.24288e+04, atom= 2031
Step=  684, Dmax= 1.7e-02 nm, Epot= -6.06542e+05 Fmax= 1.38728e+04, atom= 2031
Step=  686, Dmax= 1.0e-02 nm, Epot= -6.06654e+05 Fmax= 1.90125e+03, atom= 2031
Step=  687, Dmax= 1.2e-02 nm, Epot= -6.06664e+05 Fmax= 1.69803e+04, atom= 2031
Step=  688, Dmax= 1.5e-02 nm, Epot= -6.06811e+05 Fma

Step=  806, Dmax= 1.0e-02 nm, Epot= -6.10548e+05 Fmax= 6.23143e+03, atom= 2031
Step=  808, Dmax= 6.1e-03 nm, Epot= -6.10582e+05 Fmax= 3.17197e+03, atom= 2031
Step=  809, Dmax= 7.3e-03 nm, Epot= -6.10599e+05 Fmax= 8.09938e+03, atom= 2031
Step=  810, Dmax= 8.8e-03 nm, Epot= -6.10638e+05 Fmax= 5.43848e+03, atom= 2031
Step=  812, Dmax= 5.3e-03 nm, Epot= -6.10669e+05 Fmax= 2.68238e+03, atom= 2031
Step=  813, Dmax= 6.3e-03 nm, Epot= -6.10692e+05 Fmax= 7.08101e+03, atom= 2031
Step=  814, Dmax= 7.6e-03 nm, Epot= -6.10727e+05 Fmax= 4.62045e+03, atom= 2031
Step=  815, Dmax= 9.1e-03 nm, Epot= -6.10732e+05 Fmax= 9.42430e+03, atom= 2031
Step=  816, Dmax= 1.1e-02 nm, Epot= -6.10771e+05 Fmax= 7.42324e+03, atom= 2031
Step=  818, Dmax= 6.5e-03 nm, Epot= -6.10812e+05 Fmax= 2.68229e+03, atom= 2031
Step=  819, Dmax= 7.8e-03 nm, Epot= -6.10828e+05 Fmax= 9.43019e+03, atom= 2031
Step=  820, Dmax= 9.4e-03 nm, Epot= -6.10879e+05 Fmax= 5.12103e+03, atom= 2031
Step=  822, Dmax= 5.7e-03 nm, Epot= -6.10906e+05 Fma

Step=  938, Dmax= 6.5e-03 nm, Epot= -6.14012e+05 Fmax= 5.27696e+03, atom= 2031
Step=  939, Dmax= 7.8e-03 nm, Epot= -6.14027e+05 Fmax= 6.75614e+03, atom= 2031
Step=  940, Dmax= 9.3e-03 nm, Epot= -6.14043e+05 Fmax= 7.68093e+03, atom= 2031
Step=  941, Dmax= 1.1e-02 nm, Epot= -6.14050e+05 Fmax= 9.63493e+03, atom= 2031
Step=  942, Dmax= 1.3e-02 nm, Epot= -6.14057e+05 Fmax= 1.11449e+04, atom= 2031
Step=  944, Dmax= 8.1e-03 nm, Epot= -6.14130e+05 Fmax= 1.31845e+03, atom= 2031
Step=  945, Dmax= 9.7e-03 nm, Epot= -6.14159e+05 Fmax= 1.36051e+04, atom= 2031
Step=  946, Dmax= 1.2e-02 nm, Epot= -6.14256e+05 Fmax= 4.36700e+03, atom= 2031
Step=  948, Dmax= 7.0e-03 nm, Epot= -6.14270e+05 Fmax= 6.41792e+03, atom= 2031
Step=  949, Dmax= 8.4e-03 nm, Epot= -6.14291e+05 Fmax= 6.51686e+03, atom= 2031
Step=  950, Dmax= 1.0e-02 nm, Epot= -6.14296e+05 Fmax= 9.00325e+03, atom= 2031
Step=  951, Dmax= 1.2e-02 nm, Epot= -6.14313e+05 Fmax= 9.60727e+03, atom= 2031
Step=  953, Dmax= 7.2e-03 nm, Epot= -6.14370e+05 Fma

_(This'll take a minute or two)_

Run the simulation...

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [16]:
# Run a short simulation
! gmx grompp -f sim.mdp -c em.gro -p system.top -o md.tpr -maxwarn 1
! gmx mdrun -v -deffnm md

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

_(This'll take a minute or two)_

...And check out the result!

In [17]:
# Load the trajectory
mdt_traj = mdt.load("md.xtc", top="system.gro")
# Fix any PBC artifacts
mdt_traj.image_molecules(inplace=True)
# View the trajectory
view = nglview.show_mdtraj(mdt_traj)
view

NGLWidget(max_frame=10)